<a href="https://colab.research.google.com/github/DexterfreaK/GS2/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression

Logistic "regression" is classification algorithm. You can use it to classify things:

Just to recap, linear regression is simple: you take your data, and plot it on a graph. Then you draw a line that fits your data pretty well. Now you can use that line to predict things.

The best line is the one that minimizes a cost. For linear regression, the cost was the distance from each point of data to the line.

Similarly, when you are trying to classify something, first you plot your data on a graph.
Now you draw a line and assume anything to the left of the line belongs to one class and other half belongs to another class. This line is called a decision boundary, and it splits the data into different classes.


In the linear regression example, we predicted a number. For logistic regression, we predict a probability, like "there's a x% chance that this belongs to one of the class".

In linear regression, the prediction formula looked like this:
h<sub>$\theta$</sub> = $\theta$<sub>0</sub> + $\theta$<sub>1</sub>x

You would calculate values for $\theta$<sub>0</sub> and $\theta$<sub>1</sub>, and then use them to make a prediction.

Once you had the optimal values for  $\theta$<sub>0</sub> and  $\theta$<sub>1</sub>, the prediction part was easy.

We use almost the same formula for logistic regression, with one change:
h<sub>$\theta$</sub>(x) = g($\theta$<sub>0</sub> + $\theta$<sub>1</sub>x)





The g(...) is called the sigmoid function. This function is what makes the prediction function output a percentage. The sigmoid function constrains the result to between 0 and 1.

The sigmoid function is:

g(z) = $\frac{1}{1 + e^{-x}}$. Just like linear regression, we will use a cost function to find good values for $\theta$<sub>0</sub> and $\theta$<sub>1</sub>.

The cost was based on how far off the line was from our data points.

In logistic regression, the cost again depends on how far off our predictions are from our actual data. But we are using percentages, so the cost is calculated a little differently i.e. we calculate it using a log scale.

Cost function of linear regression is:

J($\theta$<sub>0</sub> + $\theta$<sub>1</sub>x) = ${\frac{1}{2m}}$  $\sum_{i=1}^{m} ((h_\theta(x_i) - (y_i))^2$

However, the cost function of logistic regression is:

J($\theta$) = -${\frac{1}{m}}$  $\sum_{i=1}^{m} y_i.log(h_\theta(x_i)) + (1-y_i).log(1-h_\theta(x_i))$.
This works because one of those two will always be zero, so only the other one will get used. i.e.

<br>
$y_i.log(h_\theta(x_i))$ = 0 if $y_i = 0$                  <br>
or
<br>
$(1-y_i).log(1-h_\theta(x_i))$ = 1 if $y_i = 1$.

And then we still use gradient descent to iteratively find the correct values for $\theta$

### Logistic Regression with Newton's 2nd Order optimization Method as well as the regular batch gradient descent.

We will be using the Breast Cancer Wisconsin data Set to classify if a tumour is malignant or benign, based on 30 features, such as the mean radius.

In [ ]:
## Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data Preparation
We will begin by doing some simple data cleaning and preparation before delving into the logistic regression and newton's method formulas.

In [ ]:
data = pd.read_csv('data.csv')

## Dropping a unused fields
fields_to_drop = ['id', 'Unnamed: 32']
data = data.drop(fields_to_drop, axis=1)

## Converting diagnosis to int - 1 for malignant, 0 - for benign
d = {'M': 1, 'B': 0}
data['diagnosis'] = data['diagnosis'].map(d)

## Visualising the data set
data.head()

In [ ]:
## Using 10% of dataset for testing, 10% for Validation
test_split_idx = int(data.shape[0]*0.9)
val_split_idx = int(data.shape[0]*0.8)

test_data = data[test_split_idx:]
val_data = data[val_split_idx:test_split_idx]
data = data[:val_split_idx]

## Separating data to features and targets
train_Y, train_X = data['diagnosis'], data.drop('diagnosis', axis=1)
val_Y, val_X = val_data['diagnosis'], val_data.drop('diagnosis', axis=1)
test_Y, test_X = test_data['diagnosis'], test_data.drop('diagnosis', axis=1)

In [ ]:
train_X.head()

## Logistic Regression
Logistic Regression is a method in machine learning for classification problems to output discrete values. For example, given an input of the tumour size, the logistic function can classify it as malignant (1) or benign (0).

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

## Newton's Method
Newton's method is a second-order optimization algorithm that can help us find the best weights in our logistic function in fewer iterations compared to batch gradient descent.

The generalization of Newton’s method to a multidimensional setting (also called the Newton-Raphson method) is given by:
![](images/newton.png)

For Logistic Regression, the Hessian is given by:

$$
Hf(\beta) = -X^TWX
$$
and the gradient is:

$$
\nabla f(\beta) = X^T(y-p)
$$
where
$$
W := \text{diag}\left(p(1-p)\right)
$$  
and $p$ are the predicted probabilites computed at the current value of $\beta$.


In [ ]:
def newton_step(curr, y, X, reg=None):
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T  # probability matrix - N x 1
    W = np.diag((p*(1-p))[:,0]) # N by N diagonal matrix
    hessian = X.T.dot(W).dot(X)  # 30 by 30 matrix
    grad = X.T.dot(y-p)  # 30 by 1 matrix

    # regularization step
    if reg:
        step = np.dot(np.linalg.inv(hessian + reg*np.eye(curr.shape[0])), grad)
    else:
        step = np.dot(np.linalg.inv(hessian), grad)

    beta = curr + step

    return beta

In [ ]:
def check_convergence(beta_old, beta_new, tol, iters):
    coef_change = np.abs(beta_old - beta_new)
    return not (np.any(coef_change>tol) and iters < max_iters)

In [ ]:
def test_model(X, y, beta):
    prob = np.array(sigmoid(X.dot(beta)))

    ## Converting prob to prediction, >.5 = True, <.5 = False
    prob = np.greater(prob, 0.5*np.ones((prob.shape[1],1)))
    accuracy = np.count_nonzero(np.equal(prob, y))/prob.shape[0] * 100
    return accuracy

### Training

In [ ]:
## Hyperparameters
max_iters = 20
tol=0.1 # convergence tolerance
reg_term = 1

beta_old, beta = np.ones((30,1)), np.zeros((30,1))
iter_count = 0
coefs_converged = False

while not coefs_converged:
    print('Iteration: {}'.format(iter_count))
    print('Validation Accuracy: {}%'.format(
        test_model(val_X, val_Y.to_frame(), beta_old)))
    beta_old = beta
    beta = newton_step(beta, train_Y.to_frame(), train_X, reg_term)
    iter_count += 1
    coefs_converged = check_convergence(beta_old, beta, tol, iter_count)

### Testing
As observed, using Newton's Method, the model converges in very few number of iterations and achieves a high test accuracy of 96%. Typically, Newton's method enjoys faster convergence than batch gradient descent, but each iteration takes long as finding and inverting the Hessian is computationally expensive.

In [ ]:
print('After {} Iterations'.format(iter_count))
print('Test Accuracy: {}%'.format(
        test_model(test_X, test_Y.to_frame(), beta)))

In [ ]:
beta

## Batch Gradient Ascent
To optimize the beta values of the logistic function, we can also use the following gradient ascent update rule:
![](images/sgd.png)

In [ ]:
def gd_step(curr, y, X, lr=0.0000001):
    hx = X.dot(curr)
    p = np.array(sigmoid(hx))
    change = lr * (X.T.dot(y-p))
    beta = curr + change

    return beta

In [ ]:
# Hyperparameters
batch_size = 50
lr = 0.0001
max_iters = 51

beta_old, beta = np.ones((30,1)), np.zeros((30,1))
iter_count = 0

while iter_count < max_iters:
    if iter_count % 10 == 0:
        print('Epoch: {}'.format(iter_count))
        print('Validation Accuracy: {}%'.format(
            test_model(val_X, val_Y.to_frame(), beta)))
    beta_old = beta
    for i in range(0, train_X.shape[0], batch_size):
        beta = gd_step(beta, train_Y[i:i+batch_size].to_frame(),
                        train_X[i:i+batch_size], lr)
    iter_count += 1

### Testing
While  batch gradient ascent achieves a similar accuracy of 94% after 100 iterations, a learning rate has to be determined alongside with the batch size.

In [ ]:
print('After {} Iterations'.format(iter_count))
print('Test Accuracy: {}%'.format(
        test_model(test_X, test_Y.to_frame(), beta)))

In [ ]:
beta

###  Softmax Regression

*Softmax Regression* (synonyms: *Multinomial Logistic*, *Maximum Entropy Classifier*, or just *Multi-class Logistic Regression*) is a generalization of logistic regression that we can use for multi-class classification (under the assumption that the classes are  mutually exclusive). In contrast, we use the (standard) *Logistic Regression* model in binary classification tasks.

In *Softmax Regression* (SMR), we replace the sigmoid logistic function by the so-called *softmax* function $\phi_{softmax}(\cdot)$.

$$P(y=j \mid z^{(i)}) = \phi_{softmax}(z^{(i)}) = \frac{e^{z^{(i)}}}{\sum_{j=0}^{k} e^{z_{k}^{(i)}}},$$

where we define the net input *z* as

$$z = w_1x_1 + ... + w_mx_m  + b= \sum_{l=0}^{m} w_l x_l + b= \mathbf{w}^T\mathbf{x} + b.$$

(**w** is the weight vector, $\mathbf{x}$ is the feature vector of 1 training sample, and $b$ is the bias unit.)   
Now, this softmax function computes the probability that this training sample $\mathbf{x}^{(i)}$ belongs to class $j$ given the weight and net input $z^{(i)}$. So, we compute the probability $p(y = j \mid \mathbf{x^{(i)}; w}_j)$ for each class label in  $j = 1, \ldots, k.$. Note the normalization term in the denominator which causes these class probabilities to sum up to one.

To illustrate the concept of softmax, let us walk through a concrete example. Let's assume we have a training set consisting of 4 samples from 3 different classes (0, 1, and 2)

- $x_0 \rightarrow \text{class }0$
- $x_1 \rightarrow \text{class }1$
- $x_2 \rightarrow \text{class }2$
- $x_3 \rightarrow \text{class }2$

In [ ]:
import numpy as np
y = np.array([0, 1, 2, 2])

First, we want to encode the class labels into a format that we can more easily work with; we apply one-hot encoding:

In [ ]:
y_enc = (np.arange(np.max(y) + 1) == y[:, None]).astype(float)
print('one-hot encoding:\n', y_enc)

A sample that belongs to class 0 (the first row) has a 1 in the first cell, a sample that belongs to class 2 has a 1 in the second cell of its row, and so forth.

Next, let us define the feature matrix of our 4 training samples. Here, we assume that our dataset consists of 2 features; thus, we create a 4x2 dimensional matrix of our samples and features.
Similarly, we create a 2x3 dimensional weight matrix (one row per feature and one column for each class).

In [ ]:
X = np.array([[0.1, 0.5],
              [1.1, 2.3],
              [-1.1, -2.3],
              [-1.5, -2.5]])

W = np.array([[0.1, 0.2, 0.3],
              [0.1, 0.2, 0.3]])

bias = np.array([0.01, 0.1, 0.1])

print('Inputs X:\n', X)
print('\nWeights W:\n', W)
print('\nbias:\n', bias)

To compute the net input, we multiply the 4x2 matrix feature matrix `X` with the 2x3 (n_features x n_classes) weight matrix `W`, which yields a 4x3 output matrix (n_samples x n_classes) to which we then add the bias unit:

$$\mathbf{Z} = \mathbf{X}\mathbf{W} + \mathbf{b}.$$

In [ ]:
X = np.array([[0.1, 0.5],
              [1.1, 2.3],
              [-1.1, -2.3],
              [-1.5, -2.5]])

W = np.array([[0.1, 0.2, 0.3],
              [0.1, 0.2, 0.3]])

bias = np.array([0.01, 0.1, 0.1])

print('Inputs X:\n', X)
print('\nWeights W:\n', W)
print('\nbias:\n', bias)

In [ ]:
def net_input(X, W, b):
    return (X.dot(W) + b)

net_in = net_input(X, W, bias)
print('net input:\n', net_in)

Now, it's time to compute the softmax activation that we discussed earlier:

$$P(y=j \mid z^{(i)}) = \phi_{softmax}(z^{(i)}) = \frac{e^{z^{(i)}}}{\sum_{j=0}^{k} e^{z_{k}^{(i)}}}.$$

In [ ]:
def softmax(z):
    return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

smax = softmax(net_in)
print('softmax:\n', smax)

As we can see, the values for each sample (row) nicely sum up to 1 now. E.g., we can say that the first sample   
`[ 0.29450637  0.34216758  0.36332605]` has a 29.45% probability to belong to class 0.



Now, in order to turn these probabilities back into class labels, we could simply take the argmax-index position of each row:

[[ 0.29450637  0.34216758  **0.36332605**] -> 2   
[ 0.21290077  0.32728332  **0.45981591**]  -> 2  
[ **0.42860913**  0.33380113  0.23758974]  -> 0  
[ **0.44941979**  0.32962558  0.22095463]] -> 0  

In [ ]:
def to_classlabel(z):
    return z.argmax(axis=1)

print('predicted class labels: ', to_classlabel(smax))

As we can see, our predictions are terribly wrong, since the correct class labels are `[0, 1, 2, 2]`. Now, in order to train our logistic model (e.g., via an optimization algorithm such as gradient descent), we need to define a cost function $J(\cdot)$ that we want to minimize:

$$J(\mathbf{W}; \mathbf{b}) = \frac{1}{n} \sum_{i=1}^{n} H(T_i, O_i),$$

which is the average of all cross-entropies over our $n$ training samples. The cross-entropy  function is defined as

$$H(T_i, O_i) = -\sum_m T_i \cdot log(O_i).$$

Here the $T$ stands for "target" (i.e., the *true* class labels) and the $O$ stands for output -- the computed *probability* via softmax; **not** the predicted class label.

In [ ]:
def cross_entropy(output, y_target):
    return - np.sum(np.log(output) * (y_target), axis=1)

xent = cross_entropy(smax, y_enc)
print('Cross Entropy:', xent)

In [ ]:
def cost(output, y_target):
    return np.mean(cross_entropy(output, y_target))

J_cost = cost(smax, y_enc)
print('Cost: ', J_cost)

In order to learn our softmax model -- determining the weight coefficients -- via gradient descent, we then need to compute the derivative

$$\nabla \mathbf{w}_j \, J(\mathbf{W}; \mathbf{b}).$$

I don't want to walk through the tedious details here, but this cost derivative turns out to be simply:

$$\nabla \mathbf{w}_j \, J(\mathbf{W}; \mathbf{b}) = \frac{1}{n} \sum^{n}_{i=0} \big[\mathbf{x}^{(i)}\ \big(O_i - T_i \big) \big]$$

We can then use the cost derivate to update the weights in opposite direction of the cost gradient with learning rate $\eta$:

$$\mathbf{w}_j := \mathbf{w}_j - \eta \nabla \mathbf{w}_j \, J(\mathbf{W}; \mathbf{b})$$

for each class $$j \in \{0, 1, ..., k\}$$

(note that $\mathbf{w}_j$ is the weight vector for the class $y=j$), and we update the bias units


$$\mathbf{b}_j := \mathbf{b}_j   - \eta \bigg[ \frac{1}{n} \sum^{n}_{i=0} \big(O_i - T_i  \big) \bigg].$$



As a penalty against complexity, an approach to reduce the variance of our model and decrease the degree of overfitting by adding additional bias, we can further add a regularization term such as the L2 term with the regularization parameter $\lambda$:
    
L2:        $\frac{\lambda}{2} ||\mathbf{w}||_{2}^{2}$,

where

$$||\mathbf{w}||_{2}^{2} = \sum^{m}_{l=0} \sum^{k}_{j=0} w_{i, j}$$

so that our cost function becomes

$$J(\mathbf{W}; \mathbf{b}) = \frac{1}{n} \sum_{i=1}^{n} H(T_i, O_i) + \frac{\lambda}{2} ||\mathbf{w}||_{2}^{2}$$

and we define the "regularized" weight update as

$$\mathbf{w}_j := \mathbf{w}_j -  \eta \big[\nabla \mathbf{w}_j \, J(\mathbf{W}) + \lambda \mathbf{w}_j \big].$$

(Please note that we don't regularize the bias term.)

# Softmax Regression Code

Bringing the concepts together, we could come up with an implementation as follows:

In [ ]:
import numpy as np
from time import time
#from .._base import _BaseClassifier
#from .._base import _BaseMultiClass


class SoftmaxRegression(object):

    """Softmax regression classifier.

    Parameters
    ------------
    eta : float (default: 0.01)
        Learning rate (between 0.0 and 1.0)
    epochs : int (default: 50)
        Passes over the training dataset.
        Prior to each epoch, the dataset is shuffled
        if `minibatches > 1` to prevent cycles in stochastic gradient descent.
    l2 : float
        Regularization parameter for L2 regularization.
        No regularization if l2=0.0.
    minibatches : int (default: 1)
        The number of minibatches for gradient-based optimization.
        If 1: Gradient Descent learning
        If len(y): Stochastic Gradient Descent (SGD) online learning
        If 1 < minibatches < len(y): SGD Minibatch learning
    n_classes : int (default: None)
        A positive integer to declare the number of class labels
        if not all class labels are present in a partial training set.
        Gets the number of class labels automatically if None.
    random_seed : int (default: None)
        Set random state for shuffling and initializing the weights.

    Attributes
    -----------
    w_ : 2d-array, shape={n_features, 1}
      Model weights after fitting.
    b_ : 1d-array, shape={1,}
      Bias unit after fitting.
    cost_ : list
        List of floats, the average cross_entropy for each epoch.

    """
    def __init__(self, eta=0.01, epochs=50,
                 l2=0.0,
                 minibatches=1,
                 n_classes=None,
                 random_seed=None):

        self.eta = eta
        self.epochs = epochs
        self.l2 = l2
        self.minibatches = minibatches
        self.n_classes = n_classes
        self.random_seed = random_seed

    def _fit(self, X, y, init_params=True):
        if init_params:
            if self.n_classes is None:
                self.n_classes = np.max(y) + 1
            self._n_features = X.shape[1]

            self.b_, self.w_ = self._init_params(
                weights_shape=(self._n_features, self.n_classes),
                bias_shape=(self.n_classes,),
                random_seed=self.random_seed)
            self.cost_ = []

        y_enc = self._one_hot(y=y, n_labels=self.n_classes, dtype=np.float)

        for i in range(self.epochs):
            for idx in self._yield_minibatches_idx(
                    n_batches=self.minibatches,
                    data_ary=y,
                    shuffle=True):
                # givens:
                # w_ -> n_feat x n_classes
                # b_  -> n_classes

                # net_input, softmax and diff -> n_samples x n_classes:
                net = self._net_input(X[idx], self.w_, self.b_)
                softm = self._softmax(net)
                diff = softm - y_enc[idx]
                mse = np.mean(diff, axis=0)

                # gradient -> n_features x n_classes
                grad = np.dot(X[idx].T, diff)

                # update in opp. direction of the cost gradient
                self.w_ -= (self.eta * grad +
                            self.eta * self.l2 * self.w_)
                self.b_ -= (self.eta * np.sum(diff, axis=0))

            # compute cost of the whole epoch
            net = self._net_input(X, self.w_, self.b_)
            softm = self._softmax(net)
            cross_ent = self._cross_entropy(output=softm, y_target=y_enc)
            cost = self._cost(cross_ent)
            self.cost_.append(cost)
        return self

    def fit(self, X, y, init_params=True):
        """Learn model from training data.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.
        init_params : bool (default: True)
            Re-initializes model parametersprior to fitting.
            Set False to continue training with weights from
            a previous model fitting.

        Returns
        -------
        self : object

        """
        if self.random_seed is not None:
            np.random.seed(self.random_seed)
        self._fit(X=X, y=y, init_params=init_params)
        self._is_fitted = True
        return self

    def _predict(self, X):
        probas = self.predict_proba(X)
        return self._to_classlabels(probas)

    def predict(self, X):
        """Predict targets from X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        target_values : array-like, shape = [n_samples]
          Predicted target values.

        """
        if not self._is_fitted:
            raise AttributeError('Model is not fitted, yet.')
        return self._predict(X)

    def predict_proba(self, X):
        """Predict class probabilities of X from the net input.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        Class probabilties : array-like, shape= [n_samples, n_classes]

        """
        net = self._net_input(X, self.w_, self.b_)
        softm = self._softmax(net)
        return softm

    def _net_input(self, X, W, b):
        return (X.dot(W) + b)

    def _softmax(self, z):
        return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

    def _cross_entropy(self, output, y_target):
        return - np.sum(np.log(output) * (y_target), axis=1)

    def _cost(self, cross_entropy):
        L2_term = self.l2 * np.sum(self.w_ ** 2)
        cross_entropy = cross_entropy + L2_term
        return 0.5 * np.mean(cross_entropy)

    def _to_classlabels(self, z):
        return z.argmax(axis=1)

    def _init_params(self, weights_shape, bias_shape=(1,), dtype='float64',
                     scale=0.01, random_seed=None):
        """Initialize weight coefficients."""
        if random_seed:
            np.random.seed(random_seed)
        w = np.random.normal(loc=0.0, scale=scale, size=weights_shape)
        b = np.zeros(shape=bias_shape)
        return b.astype(dtype), w.astype(dtype)

    def _one_hot(self, y, n_labels, dtype):
        """Returns a matrix where each sample in y is represented
           as a row, and each column represents the class label in
           the one-hot encoding scheme.

        Example:

            y = np.array([0, 1, 2, 3, 4, 2])
            mc = _BaseMultiClass()
            mc._one_hot(y=y, n_labels=5, dtype='float')

            np.array([[1., 0., 0., 0., 0.],
                      [0., 1., 0., 0., 0.],
                      [0., 0., 1., 0., 0.],
                      [0., 0., 0., 1., 0.],
                      [0., 0., 0., 0., 1.],
                      [0., 0., 1., 0., 0.]])

        """
        mat = np.zeros((len(y), n_labels))
        for i, val in enumerate(y):
            mat[i, val] = 1
        return mat.astype(dtype)

    def _yield_minibatches_idx(self, n_batches, data_ary, shuffle=True):
            indices = np.arange(data_ary.shape[0])

            if shuffle:
                indices = np.random.permutation(indices)
            if n_batches > 1:
                remainder = data_ary.shape[0] % n_batches

                if remainder:
                    minis = np.array_split(indices[:-remainder], n_batches)
                    minis[-1] = np.concatenate((minis[-1],
                                                indices[-remainder:]),
                                               axis=0)
                else:
                    minis = np.array_split(indices, n_batches)

            else:
                minis = (indices,)

            for idx_batch in minis:
                yield idx_batch

    def _shuffle_arrays(self, arrays):
        """Shuffle arrays in unison."""
        r = np.random.permutation(len(arrays[0]))
        return [ary[r] for ary in arrays]

## Example 1 - Gradient Descent

In [ ]:
from mlxtend.data import iris_data
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt

# Loading Data

X, y = iris_data()
X = X[:, [0, 3]] # sepal length and petal width

# standardize
X[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()

lr = SoftmaxRegression(eta=0.01, epochs=10, minibatches=1, random_seed=0)
lr.fit(X, y)

plot_decision_regions(X, y, clf=lr)
plt.title('Softmax Regression - Gradient Descent')
plt.show()

plt.plot(range(len(lr.cost_)), lr.cost_)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

Continue training for another 800 epochs by calling the `fit` method with `init_params=False`.

In [ ]:
lr.epochs = 800

lr.fit(X, y, init_params=False)

plot_decision_regions(X, y, clf=lr)
plt.title('Softmax Regression - Stochastic Gradient Descent')
plt.show()

plt.plot(range(len(lr.cost_)), lr.cost_)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

### Predicting Class Labels

In [ ]:
y_pred = lr.predict(X)
print('Last 3 Class Labels: %s' % y_pred[-3:])

### Predicting Class Probabilities

In [ ]:
y_pred = lr.predict_proba(X)
print('Last 3 Class Labels:\n %s' % y_pred[-3:])

## Example 2 - Stochastic Gradient Descent

In [ ]:
from mlxtend.data import iris_data
from mlxtend.plotting import plot_decision_regions
from mlxtend.classifier import SoftmaxRegression
import matplotlib.pyplot as plt

# Loading Data

X, y = iris_data()
X = X[:, [0, 3]] # sepal length and petal width

# standardize
X[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()

lr = SoftmaxRegression(eta=0.05, epochs=200, minibatches=len(y), random_seed=0)
lr.fit(X, y)

plot_decision_regions(X, y, clf=lr)
plt.title('Softmax Regression - Stochastic Gradient Descent')
plt.show()

plt.plot(range(len(lr.cost_)), lr.cost_)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

# **Exercise**

Load palmerpenguin dataset which contains data for 344 penguins. There are 3 different species of penguins in this dataset, collected from 3 islands in the Palmer Archipelago, Antarctica.

In the dataset, culmen is the upper ridge of a bird’s bill. In the simplified penguins data, culmen length and depth are renamed as variables bill_length_mm and bill_depth_mm to be more intuitive.

Firstly, perform the required preprocessing and then apply the logistic regression technique. Then train logistic model with different optimization algorithm by minimizing errors between predicted and actual results and compare the result.

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('penguin.csv')
# Check for missing values
print(df.isnull().sum())
# Drop rows with missing values
df.dropna(inplace=True)


island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
species               0
dtype: int64


In [2]:
from sklearn.preprocessing import LabelEncoder

# Encode 'species' using LabelEncoder
le_species = LabelEncoder()
df['species'] = le_species.fit_transform(df['species'])

# Encode 'sex' using LabelEncoder
le_sex = LabelEncoder()
df['sex'] = le_sex.fit_transform(df['sex'])

# One-hot encode 'island'
df = pd.get_dummies(df, columns=['island'])


In [3]:
from sklearn.preprocessing import StandardScaler

# List of numerical features
numerical_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

# Scale numerical features
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])


In [4]:
from sklearn.model_selection import train_test_split

# Define features and target variable
X = df.drop('species', axis=1)
y = df['species']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

# List of solvers to compare
solvers = ['newton-cg', 'lbfgs', 'sag', 'saga']

# Dictionary to store results
results = {}

for solver in solvers:
    model = LogisticRegression(
        solver=solver, multi_class='multinomial', max_iter=1000
    )
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[solver] = {'Accuracy': acc, 'Training Time (s)': training_time}
    print(f"Solver: {solver}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Training Time: {training_time:.4f} seconds")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le_species.classes_))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("-" * 50)


Solver: newton-cg
Accuracy: 1.0000
Training Time: 0.0211 seconds
Classification Report:
              precision    recall  f1-score   support

      Adelie       1.00      1.00      1.00        29
   Chinstrap       1.00      1.00      1.00        14
      Gentoo       1.00      1.00      1.00        24

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67

Confusion Matrix:
[[29  0  0]
 [ 0 14  0]
 [ 0  0 24]]
--------------------------------------------------
Solver: lbfgs
Accuracy: 1.0000
Training Time: 0.0165 seconds
Classification Report:
              precision    recall  f1-score   support

      Adelie       1.00      1.00      1.00        29
   Chinstrap       1.00      1.00      1.00        14
      Gentoo       1.00      1.00      1.00        24

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg   

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and wi

In [6]:
# Create a DataFrame from the results
results_df = pd.DataFrame(results).T
print("Comparison of Different Solvers:")
print(results_df)


Comparison of Different Solvers:
           Accuracy  Training Time (s)
newton-cg       1.0           0.021144
lbfgs           1.0           0.016504
sag             1.0           0.009618
saga            1.0           0.009632
